# Project 2: Panoramic Image Stitching

This is Project 2 for [UW CSE P576 Computer Vision](https://courses.cs.washington.edu/courses/csep576/18sp). 

**Getting Started:** You should complete **[Project 1](https://courses.cs.washington.edu/courses/csep57
6/18sp/projects/Project1.html "Project 1")** first (you will need interest points and descriptors from this project). The source files for both projects are [here](https://courses.cs.washington.edu/courses/csep576/18sp/projects/project12/project12.zip "Project 1 and 2 Source Files"). To run the project locally you will need IPython/Jupyter installed, e.g., see instructions at http://jupyter.org/install.html. Launch Jupyter and open `Project2.ipynb`. Alternatively, you can import the standalone version of the notebook into [Colaboratory](https://colab.research.google.com "Colab") and run it without installing anything. Use File->Upload Notebook in Colab and open the notebook in `standalone/Project2s.ipynb`.

**This project:** In this project you will implement a panoramic image stitcher. This will build on the interest points and descriptors developed in Project 1. You'll begin with geometric filtering via RANSAC, then estimate pairwise rotations and chain these together to align the panorama. When you have a basic stitcher working, improve it with better alignment, blending, or other new features and document your findings.

**What to turn in:** Turn in a pdf or static html copy of your completed ipynb notebook as well as the source .ipynb and any source .py files that you modified. Clearly describe any enhancements or experiments you tried in your ipynb notebook.

In [ ]:
import numpy as np
import scipy.linalg
import os.path
from time import time
import types
import matplotlib.pyplot as plt

#import im_util
#import interest_point
#import ransac
#import geometry
#import render
#import panorama

%matplotlib inline
# edit this line to change the figure size
plt.rcParams['figure.figsize'] = (16.0, 10.0)
# force auto-reload of import modules before running code 
%load_ext autoreload
%autoreload 2

In [ ]:
!wget -nc https://courses.cs.washington.edu/courses/csep576/18sp/projects/project12/pano_images.zip && unzip -n -d data pano_images.zip

### im_util.py

In [ ]:
# Copyright 2017 Google Inc.

# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at

# https://www.apache.org/licenses/LICENSE-2.0

# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

import numpy as np
import PIL.Image as pil
import scipy.signal as sps
import matplotlib.pyplot as plt
from scipy.ndimage import map_coordinates

def convolve_1d(x, k):
  """
  Convolve vector x with kernel k

  Inputs: x=input vector (Nx)
          k=input kernel (Nk)

  Outputs: y=output vector (Nx)
  """
  y=np.zeros_like(x)

  """
  *******************************************
  *** TODO: write code to perform convolution
  *******************************************

  The output should be the same size as the input
  You can assume zero padding, and an odd-sized kernel
  """


  """
  *******************************************
  """

  return y

def convolve_rows(im, k):
  """
  Convolve image im with kernel k

  Inputs: im=input image (H, W, B)
          k=1D convolution kernel (N)

  Outputs: im_out=output image (H, W, B)
  """
  im_out = np.zeros_like(im)

  """
  *****************************************
  *** TODO: write code to convolve an image
  *****************************************

  Convolve the rows of image im with kernel k
  The output should be the same size as the input
  You can assume zero padding, and an odd-sized kernel
  """


  """
  *****************************************
  """

  return im_out

def gauss_kernel(sigma):
  """
  1D Gauss kernel of standard deviation sigma
  """
  l = int(np.ceil(2 * sigma))
  x = np.linspace(-l, l, 2*l+1)

  # FORNOW
  gx = np.zeros_like(x)

  """
  *******************************************
  *** TODO: compute gaussian kernel at each x
  *******************************************
  """


  """
  *******************************************
  """

  gx = np.expand_dims(gx,0)
  return gx

def convolve_gaussian(im, sigma):
  """
  2D gaussian convolution
  """
  imc=np.zeros_like(im)

  """
  ***************************************
  *** TODO separable gaussian convolution
  ***************************************
  """


  """
  ***************************************
  """
  return imc

def compute_gradients(img):

  Ix=np.zeros_like(img)
  Iy=np.zeros_like(img)

  """
  ***********************************************
  *** TODO: write code to compute image gradients
  ***********************************************
  """


  """
  ***********************************************
  """
  return Ix, Iy

def image_open(filename):
  """
  Returns a numpy float image with values in the range (0,1)
  """
  pil_im = pil.open(filename)
  im_np = np.array(pil_im).astype(np.float32)
  im_np /= 255.0
  return im_np

def image_save(im_np, filename):
  """
  Saves a numpy float image to file
  """
  if (len(im_np.shape)==2):
    im_np = np.expand_dims(im_np, 2)
  if (im_np.shape[2]==1):
    im_np= np.repeat(im_np, 3, axis=2)
  im_np = np.maximum(0.0, np.minimum(im_np, 1.0))
  pil_im = pil.fromarray((im_np*255).astype(np.uint8))
  pil_im.save(filename)

def image_figure(im, dpi=100):
  """
  Creates a matplotlib figure around an image,
  useful for writing to file with savefig()
  """
  H,W,_=im.shape
  fig=plt.figure()
  fig.set_size_inches(W/dpi, H/dpi)
  ax=fig.add_axes([0,0,1,1])
  ax.imshow(im)
  return fig, ax

def plot_two_images(im1, im2):
  """
  Plot two images and return axis handles
  """
  ax1=plt.subplot(1,2,1)
  plt.imshow(im1)
  plt.axis('off')
  ax2=plt.subplot(1,2,2)
  plt.imshow(im2)
  plt.axis('off')
  return ax1, ax2

def normalise_01(im):
  """
  Normalise image to the range (0,1)
  """
  mx = im.max()
  mn = im.min()
  den = mx-mn
  small_val = 1e-9
  if (den < small_val):
    print('image normalise_01 -- divisor is very small')
    den = small_val
  return (im-mn)/den

def grey_to_rgb(img):
  """
  Convert greyscale to rgb image
  """
  if (len(img.shape)==2):
    img = np.expand_dims(img, 2)

  img3 = np.repeat(img, 3, 2)
  return img3

def disc_mask(l):
  """
  Create a binary cirular mask of radius l
  """
  sz = 2 * l + 1
  m = np.zeros((sz,sz))
  x = np.linspace(-l,l,2*l+1)/l
  x = np.expand_dims(x, 1)
  m = x**2
  m = m + m.T
  m = m<1
  m = np.expand_dims(m, 2)
  return m

def convolve(im, kernel):
  """
  Wrapper for scipy convolution function
  This implements a general 2D convolution of image im with kernel
  Note that strictly speaking this is correlation not convolution

  Inputs: im=input image (H, W, B) or (H, W)
          kernel=kernel (kH, kW)

  Outputs: imc=output image (H, W, B)
  """
  if (len(im.shape)==2):
    im = np.expand_dims(im, 2)
  H, W, B = im.shape
  imc = np.zeros((H, W, B))
  for band in range(B):
    imc[:, :, band] = sps.correlate2d(im[:, :, band], kernel, mode='same')
  return imc

def coordinate_image(num_rows,num_cols,r0,r1,c0,c1):
  """
  Creates an image size num_rows, num_cols
  with coordinates linearly spaced in from r0->r1 and c0->c1
  """
  rval=np.linspace(r0,r1,num_rows)
  cval=np.linspace(c0,c1,num_cols)
  c,r=np.meshgrid(cval,rval)
  M = np.stack([r,c,np.ones(r.shape)],-1)
  return M

def transform_coordinates(coord_image, M):
  """
  Transform an image containing row,col,1 coordinates by matrix M
  """
  M=np.expand_dims(M,2)
  uh=np.dot(coord_image,M.T)
  uh=uh[:, :, 0, :]
  uh=uh/np.expand_dims(uh[:, :, 2],2)
  return uh

def warp_image(im, coords):
  """
  Warp image im using row,col,1 image coords
  """
  im_rows,im_cols,im_bands=im.shape
  warp_rows,warp_cols,_=coords.shape
  map_coords=np.zeros((3,warp_rows,warp_cols,im_bands))
  for b in range(im_bands):
    map_coords[0,:,:,b]=coords[:,:,0]
    map_coords[1,:,:,b]=coords[:,:,1]
    map_coords[2,:,:,b]=b
  warp_im = map_coordinates(im, map_coords, order=1)
  return warp_im

# allow accessing these functions by im_util.*
im_util=types.SimpleNamespace()
im_util.convolve_1d=convolve_1d
im_util.convolve_rows=convolve_rows
im_util.gauss_kernel=gauss_kernel
im_util.convolve_gaussian=convolve_gaussian
im_util.compute_gradients=compute_gradients
im_util.image_open=image_open
im_util.image_save=image_save
im_util.image_figure=image_figure
im_util.plot_two_images=plot_two_images
im_util.normalise_01=normalise_01
im_util.grey_to_rgb=grey_to_rgb
im_util.disc_mask=disc_mask
im_util.convolve=convolve
im_util.coordinate_image=coordinate_image
im_util.transform_coordinates=transform_coordinates
im_util.warp_image=warp_image

### interest_point.py

In [ ]:
# Copyright 2017 Google Inc.

# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at

# https://www.apache.org/licenses/LICENSE-2.0

# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

import numpy as np
import scipy.ndimage.filters as filters
from scipy.ndimage import map_coordinates
from matplotlib.patches import Circle
import matplotlib.pyplot as plt

#import im_util

class InterestPointExtractor:
  """
  Class to extract interest points from an image
  """
  def __init__(self):
    self.params={}
    self.params['border_pixels']=10
    self.params['strength_threshold_percentile']=95
    self.params['supression_radius_frac']=0.01

  def find_interest_points(self, img):
    """
    Find interest points in greyscale image img

    Inputs: img=greyscale input image (H, W, 1)

    Outputs: ip=interest points of shape (2, N)
    """
    ip_fun = self.corner_function(img)
    row, col = self.find_local_maxima(ip_fun)

    ip = np.stack((row,col))
    return ip

  def corner_function(self, img):
    """
    Compute corner strength function in image im

    Inputs: img=grayscale input image (H, W, 1)

    Outputs: ip_fun=interest point strength function (H, W, 1)
    """

    H, W, _ = img.shape

    # FORNOW: random interest point function
    ip_fun = np.random.randn(H, W, 1)

    """
    **********************************************************
    *** TODO: write code to compute a corner strength function
    **********************************************************
    """


    """
    **********************************************************
    """

    return ip_fun

  def find_local_maxima(self, ip_fun):
    """
    Find local maxima in interest point strength function

    Inputs: ip_fun=corner strength function (H, W, 1)

    Outputs: row,col=coordinates of interest points
    """

    H, W, _ = ip_fun.shape

    # radius for non-maximal suppression
    suppression_radius_pixels = int(self.params['supression_radius_frac']*max(H, W))

    # minimum of strength function for corners
    strength_threshold=np.percentile(ip_fun, self.params['strength_threshold_percentile'])

    # don't return interest points within border_pixels of edge
    border_pixels = self.params['border_pixels']

    # row and column coordinates of interest points
    row = []
    col = []

    # FORNOW: random row and column coordinates
    row = np.random.randint(0,H,100)
    col = np.random.randint(0,W,100)

    """
    ***************************************************
    *** TODO: write code to find local maxima in ip_fun
    ***************************************************

    Hint: try scipy filters.maximum_filter with im_util.disc_mask
    """


    """
    ***************************************************
    """

    return row, col

class DescriptorExtractor:
  """
  Extract descriptors around interest points
  """
  def __init__(self):
    self.params={}
    self.params['patch_size']=8
    self.params['ratio_threshold']=1.0

  def get_descriptors(self, img, ip):
    """
    Extact descriptors from grayscale image img at interest points ip

    Inputs: img=grayscale input image (H, W, 1)
            ip=interest point coordinates (2, N)

    Returns: descriptors=vectorized descriptors (N, num_dims)
    """
    patch_size=self.params['patch_size']
    patch_size_div2=int(patch_size/2)
    num_dims=patch_size**2

    H,W,_=img.shape
    num_ip=ip.shape[1]
    descriptors=np.zeros((num_ip,num_dims))


    for i in range(num_ip):
      row=ip[0,i]
      col=ip[1,i]

      # FORNOW: random image patch
      patch=np.random.randn(patch_size,patch_size)

      """
      ******************************************************
      *** TODO: write code to extract descriptor at row, col
      ******************************************************
      """


      """
      ******************************************************
      """

      descriptors[i, :]=np.reshape(patch,num_dims)

    # normalise descriptors to 0 mean, unit length
    mn=np.mean(descriptors,1,keepdims=True)
    sd=np.std(descriptors,1,keepdims=True)
    small_val = 1e-6
    descriptors = (descriptors-mn)/(sd+small_val)

    return descriptors

  def compute_distances(self, desc1, desc2):
    """
    Compute distances between descriptors

    Inputs: desc1=descriptor array (N1, num_dims)
            desc2=descriptor array (N2, num_dims)

    Returns: dists=array of distances (N1,N2)
    """
    N1,num_dims=desc1.shape
    N2,num_dims=desc2.shape

    ATB=np.dot(desc1,desc2.T)
    AA=np.sum(desc1*desc1,1)
    BB=np.sum(desc2*desc2,1)

    dists=-2*ATB+np.expand_dims(AA,1)+BB

    return dists

  def match_descriptors(self, desc1, desc2):
    """
    Find nearest neighbour matches between descriptors

    Inputs: desc1=descriptor array (N1, num_dims)
            desc2=descriptor array (N2, num_dims)

    Returns: match_idx=nearest neighbour index for each desc1 (N1)
    """
    dists=self.compute_distances(desc1, desc2)

    match_idx=np.argmin(dists,1)

    return match_idx

  def match_ratio_test(self, desc1, desc2):
    """
    Find nearest neighbour matches between descriptors
    and perform ratio test

    Inputs: desc1=descriptor array (N1, num_dims)
            desc2=descriptor array (N2, num_dims)

    Returns: match_idx=nearest neighbour inded for each desc1 (N1)
             ratio_pass=whether each match passes ratio test (N1)
    """
    N1,num_dims=desc1.shape

    dists=self.compute_distances(desc1, desc2)

    sort_idx=np.argsort(dists,1)

    #match_idx=np.argmin(dists,1)
    match_idx=sort_idx[:,0]

    d1NN=dists[np.arange(0,N1),sort_idx[:,0]]
    d2NN=dists[np.arange(0,N1),sort_idx[:,1]]

    ratio_threshold=self.params['ratio_threshold']
    ratio_pass=(d1NN<ratio_threshold*d2NN)

    return match_idx,ratio_pass

def draw_interest_points_ax(ip, ax):
  """
  Draw interest points ip on axis ax
  """
  for row,col in zip(ip[0,:],ip[1,:]):
    circ1 = Circle((col,row), 5)
    circ1.set_color('black')
    circ2 = Circle((col,row), 3)
    circ2.set_color('white')
    ax.add_patch(circ1)
    ax.add_patch(circ2)

def draw_interest_points_file(im, ip, filename):
  """
  Draw interest points ip on image im and save to filename
  """
  fig,ax = im_util.image_figure(im)
  draw_interest_points_ax(ip, ax)
  fig.savefig(filename)
  plt.close(fig)

def draw_matches_ax(ip1, ipm, ax1, ax2):
  """
  Draw matches ip1, ipm on axes ax1, ax2
  """
  for r1,c1,r2,c2 in zip(ip1[0,:], ip1[1,:], ipm[0,:], ipm[1,:]):
    rand_colour=np.random.rand(3,)

    circ1 = Circle((c1,r1), 5)
    circ1.set_color('black')
    circ2 = Circle((c1,r1), 3)
    circ2.set_color(rand_colour)
    ax1.add_patch(circ1)
    ax1.add_patch(circ2)

    circ3 = Circle((c2,r2), 5)
    circ3.set_color('black')
    circ4 = Circle((c2,r2), 3)
    circ4.set_color(rand_colour)
    ax2.add_patch(circ3)
    ax2.add_patch(circ4)

def draw_matches_file(im1, im2, ip1, ipm, filename):
  """
  Draw matches ip1, ipm on images im1, im2 and save to filename
  """
  H1,W1,B1=im1.shape
  H2,W2,B2=im2.shape

  im3 = np.zeros((max(H1,H2),W1+W2,3))
  im3[0:H1,0:W1,:]=im1
  im3[0:H2,W1:(W1+W2),:]=im2

  fig,ax = im_util.image_figure(im3)
  col_offset=W1

  for r1,c1,r2,c2 in zip(ip1[0,:], ip1[1,:], ipm[0,:], ipm[1,:]):
    rand_colour=np.random.rand(3,)

    circ1 = Circle((c1,r1), 5)
    circ1.set_color('black')
    circ2 = Circle((c1,r1), 3)
    circ2.set_color(rand_colour)
    ax.add_patch(circ1)
    ax.add_patch(circ2)

    circ3 = Circle((c2+col_offset,r2), 5)
    circ3.set_color('black')
    circ4 = Circle((c2+col_offset,r2), 3)
    circ4.set_color(rand_colour)
    ax.add_patch(circ3)
    ax.add_patch(circ4)

  fig.savefig(filename)
  plt.close(fig)

def plot_descriptors(desc,plt):
  """
  Plot a random set of descriptor patches
  """
  num_ip,num_dims = desc.shape
  patch_size = int(np.sqrt(num_dims))

  N1,N2=2,8
  figsize0=plt.rcParams['figure.figsize']
  plt.rcParams['figure.figsize'] = (16.0, 4.0)
  for i in range(N1):
    for j in range(N2):
      ax=plt.subplot(N1,N2,i*N2+j+1)
      rnd=np.random.randint(0,num_ip)
      desc_im=np.reshape(desc[rnd,:],(patch_size,patch_size))
      plt.imshow(im_util.grey_to_rgb(im_util.normalise_01(desc_im)))
      plt.axis('off')

  plt.rcParams['figure.figsize']=figsize0

def plot_matching_descriptors(desc1,desc2,desc1_id,desc2_id,plt):
  """
  Plot a random set of matching descriptor patches
  """
  num_inliers=desc1_id.size
  num_ip,num_dims = desc1.shape
  patch_size=int(np.sqrt(num_dims))

  figsize0=plt.rcParams['figure.figsize']

  N1,N2=1,8
  plt.rcParams['figure.figsize'] = (16.0, N1*4.0)

  for i in range(N1):
    for j in range(N2):
      rnd=np.random.randint(0,num_inliers)

      desc1_rnd=desc1_id[rnd]
      desc2_rnd=desc2_id[rnd]

      desc1_im=np.reshape(desc1[desc1_rnd,:],(patch_size,patch_size))
      desc2_im=np.reshape(desc2[desc2_rnd,:],(patch_size,patch_size))

      ax=plt.subplot(2*N1,N2,2*i*N2+j+1)
      plt.imshow(im_util.grey_to_rgb(im_util.normalise_01(desc1_im)))
      plt.axis('off')
      ax=plt.subplot(2*N1,N2,2*i*N2+N2+j+1)
      plt.imshow(im_util.grey_to_rgb(im_util.normalise_01(desc2_im)))
      plt.axis('off')

  plt.rcParams['figure.figsize'] = figsize0

# allow accessing these functions by interest_point.*
interest_point=types.SimpleNamespace()
interest_point.InterestPointExtractor=InterestPointExtractor
interest_point.DescriptorExtractor=DescriptorExtractor
interest_point.draw_interest_points_ax=draw_interest_points_ax
interest_point.draw_interest_points_file=draw_interest_points_file
interest_point.draw_matches_ax=draw_matches_ax
interest_point.draw_matches_file=draw_matches_file
interest_point.plot_descriptors=plot_descriptors
interest_point.plot_matching_descriptors=plot_matching_descriptors

### ransac.py

In [ ]:
# Copyright 2017 Google Inc.

# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at

# https://www.apache.org/licenses/LICENSE-2.0

# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

import os.path
import numpy as np
from time import time

#import im_util
#import interest_point
#import geometry

class RANSAC:
  """
  Find 2-view consistent matches using RANSAC
  """
  def __init__(self):
    self.params={}
    self.params['num_iterations']=500
    self.params['inlier_dist']=10
    self.params['min_sample_dist']=2

  def consistent(self, H, p1, p2):
    """
    Find interest points that are consistent with 2D transform H

    Inputs: H=homography matrix (3,3)
            p1,p2=corresponding points in images 1,2 of shape (2, N)

    Outputs: cons=list of inliers indicated by true/false (num_points)

    Assumes that H maps from 1 to 2, i.e., hom(p2) ~= H hom(p1)
    """

    cons = np.zeros((p1.shape[1]))
    inlier_dist = self.params['inlier_dist']

    """
    ************************************************
    *** TODO: write code to check consistency with H
    ************************************************
    """


    """
    ************************************************
    """

    return cons

  def compute_similarity(self,p1,p2):
    """
    Compute similarity transform between pairs of points

    Input: p1,p2=arrays of coordinates (2, 2)

    Output: Similarity matrix S (3, 3)

    Assume S maps from 1 to 2, i.e., hom(p2) = S hom(p1)
    """

    S = np.eye(3,3)

    """
    ****************************************************
    *** TODO: write code to compute similarity transform
    ****************************************************
    """


    """
    ****************************************************
    """

    return S

  def ransac_similarity(self, ip1, ipm):
    """
    Find 2-view consistent matches under a Similarity transform

    Inputs: ip1=interest points (2, num_points)
            ipm=matching interest points (2, num_points)
            ip[0,:]=row coordinates, ip[1, :]=column coordinates

    Outputs: S_best=Similarity matrix (3,3)
             inliers_best=list of inliers indicated by true/false (num_points)
    """
    S_best=np.eye(3,3)
    inliers_best=[]

    """
    *****************************************************
    *** TODO: use ransac to find a similarity transform S
    *****************************************************
    """


    """
    *****************************************************
    """

    return S_best, inliers_best

# allow accessing these functions by ransac.*
ransac=types.SimpleNamespace()
ransac.RANSAC=RANSAC

### match.py

In [ ]:
# Copyright 2017 Google Inc.

# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at

# https://www.apache.org/licenses/LICENSE-2.0

# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

import os.path
from time import time
import numpy as np

#import interest_point
#import ransac

class ImageMatcher:
  """
  Find geometrically consistent matches in a set of images
  """
  def __init__(self, params={}):
    self.params=params
    self.params.setdefault('draw_interest_points', False)
    self.params.setdefault('draw_matches', False)
    self.params.setdefault('results_dir', os.path.expanduser('~/results/tmp/'))

  def match_images(self, images):
    """
    Find geometrically consistent matches between images
    """
    # extract interest points and descriptors
    print('[ find interest points ]')
    t0=time()
    interest_points=[]
    descriptors=[]
    ip_ex = interest_point.InterestPointExtractor()
    desc_ex = interest_point.DescriptorExtractor()
    num_images = len(images)

    for i in range(num_images):
      im = images[i]
      img = np.mean(im, 2, keepdims=True)
      ip = ip_ex.find_interest_points(img)
      print(' found '+str(ip.shape[1])+' interest points')
      interest_points.append(ip)
      desc = desc_ex.get_descriptors(img, ip)
      descriptors.append(desc)
      if (self.params['draw_interest_points']):
        interest_point.draw_interest_points_file(im, ip, self.params['results_dir']+'/ip'+str(i)+'.jpg')

    t1=time()
    print(' % .2f secs ' % (t1-t0))

    # match descriptors and perform ransac
    print('[ match descriptors ]')
    matches = [[None]*num_images for _ in range(num_images)]
    num_matches = np.zeros((num_images, num_images))

    t0=time()
    rn = ransac.RANSAC()

    for i in range(num_images):
      ipi = interest_points[i]
      print(' image '+str(i))
      for j in range(num_images):
        if (i==j):
          continue

        matchesij = desc_ex.match_descriptors(descriptors[i],descriptors[j])
        ipm = interest_points[j][:, matchesij]
        S, inliers = rn.ransac_similarity(ipi, ipm)
        num_matches[i,j]=np.sum(inliers)
        ipic=ipi[:, inliers]
        ipmc=ipm[:, inliers]
        matches[i][j]=np.concatenate((ipic,ipmc),0)

        if (self.params['draw_matches']):
          imi = images[i]
          imj = images[j]
          interest_point.draw_matches_file(imi, imj, ipi, ipm, self.params['results_dir']+'/match_raw_'+str(i)+str(j)+'.jpg')
          interest_point.draw_matches_file(imi, imj, ipic, ipmc, self.params['results_dir']+'/match_'+str(i)+str(j)+'.jpg')

    t1=time()
    print(' % .2f secs' % (t1-t0))

    return matches, num_matches

# allow accessing these functions by match.*
match=types.SimpleNamespace()
match.ImageMatcher=ImageMatcher

### geometry.py

In [ ]:
# Copyright 2017 Google Inc.

# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at

# https://www.apache.org/licenses/LICENSE-2.0

# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

import numpy as np

def compute_rotation(ip1, ip2, K1, K2):
  """
  Find rotation matrix R such that |r2 - R*r1|^2 is minimised

  Inputs: ip1,ip2=corresponding interest points (2,num_points),
          K1,K2=camera calibration matrices (3,3)

  Outputs: R=rotation matrix R (3,3)

  r1,r2 are corresponding rays (unit normalised camera coordinates) in image 1,2
  """

  R=H=np.eye(3,3)

  """
  **********************************************************************
  *** TODO: write code to compute 3D rotation between corresponding rays
  **********************************************************************
  """


  """
  **********************************************************************
  """
  return R, H

def get_calibration(imshape, fov_degrees):
  """
  Return calibration matrix K given image shape and field of view

  See note on calibration matrix in documentation of K(f, H, W)
  """
  H, W, _ = imshape
  f = max(H,W)/(2*np.tan((fov_degrees/2)*np.pi/180))
  K1 = K(f,H,W)
  return K1

def K(f,H,W):
  """
  Return camera calibration matrix given focal length and image size

  Inputs: f=focal length, H=image height, W=image width all in pixels

  Outputs: K=calibration matrix (3, 3)

  The calibration matrix maps camera coordinates [X,Y,Z] to homogeneous image
  coordinates ~[row,col,1]. X is assumed to point along the positive col direction,
  i.e., incrementing X increments the col dimension in the image
  """
  K1=np.zeros((3,3))
  K1[0,1]=K1[1,0]=f
  K1[0,2]=H/2
  K1[1,2]=W/2
  K1[2,2]=1
  return K1

def hom(p):
  """
  Convert points to homogeneous coordiantes
  """
  ph=np.concatenate((p,np.ones((1,p.shape[1]))))
  return ph

def unhom(ph):
  """
  Convert points from homogeneous to regular coordinates
  """
  p=ph/ph[2,:]
  p=p[0:2,:]
  return p

# allow accessing these functions by geometry.*
geometry=types.SimpleNamespace()
geometry.compute_rotation=compute_rotation
geometry.get_calibration=get_calibration
geometry.K=K
geometry.hom=hom
geometry.unhom=unhom

### render.py

In [ ]:
# Copyright 2017 Google Inc.

# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at

# https://www.apache.org/licenses/LICENSE-2.0

# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

import numpy as np
import skimage.transform
from scipy.ndimage import map_coordinates
#import im_util

def pairwise_warp(im1,im2,H):
  """
  Warp im1 to im2 coords and vice versa
  """
  # skimage transforms assume c,r rather than r,c
  P=np.zeros((3,3))
  P[0,1]=P[1,0]=P[2,2]=1

  HP=np.dot(np.dot(P,H),P)
  HPinv=np.linalg.inv(HP)

  im1_w = skimage.transform.warp(im1,skimage.transform.ProjectiveTransform(HPinv))
  im2_w = skimage.transform.warp(im2,skimage.transform.ProjectiveTransform(HP))

  return im1_w, im2_w

def pairwise_warp_file(im1,im2,H,results_prefix):
  """
  Warp im1 to im2 coords and vice versa
  """
  im1_w,im2_w = pairwise_warp(im1,im2,H)
  im_util.image_save(0.5*(im1+im2_w), results_prefix+'_im1.jpg')
  im_util.image_save(0.5*(im2+im1_w), results_prefix+'_im2.jpg')

def render_spherical(images, P_matrices, params={}):
  """
  Render images with given projection matrices in spherical coordinates
  """
  params.setdefault('theta_min', -45)
  params.setdefault('theta_max', 45)
  params.setdefault('phi_min', -30)
  params.setdefault('phi_max', 30)
  params.setdefault('render_width', 800)

  theta_min=params['theta_min'] * np.pi/180
  theta_max=params['theta_max'] * np.pi/180
  phi_min=params['phi_min'] * np.pi/180
  phi_max=params['phi_max'] * np.pi/180

  render_width=params['render_width']
  render_height=int(render_width*(phi_max-phi_min)/(theta_max-theta_min))

  world_coords=np.zeros((render_height, render_width, 3))

  theta=np.linspace(theta_min, theta_max, render_width)
  phi=np.linspace(phi_max, phi_min, render_height)

  cos_phi=np.expand_dims(np.cos(phi),1)
  sin_phi=np.expand_dims(np.sin(phi),1)
  cos_theta=np.expand_dims(np.cos(theta),0)
  sin_theta=np.expand_dims(np.sin(theta),0)

  X=np.dot(cos_phi, sin_theta)
  Y=-np.dot(sin_phi, np.ones((1,render_width)))
  Z=np.dot(cos_phi, cos_theta)

  world_coords[:, :, 0]=X
  world_coords[:, :, 1]=Y
  world_coords[:, :, 2]=Z
  wc=np.expand_dims(world_coords,2)

  pano_im=np.zeros((render_height, render_width, 4))
  im_coords=np.zeros((3,render_height, render_width,4))

  for im,P in zip(images,P_matrices):
    # compute coordinates u ~ P [X Y Z]
    uh=np.dot(wc,P.T)
    uh=uh[:, :, 0, :]
    uh=uh/np.expand_dims(uh[:, :, 2],2)

    for b in range(4):
      im_coords[0,:,:,b]=uh[:, :, 0]
      im_coords[1,:,:,b]=uh[:, :, 1]
      im_coords[2,:,:,b]=b

    # add alpha channel
    H,W,_=im.shape
    ima=np.concatenate((im,np.ones((H,W,1))),2)
    pano_im += map_coordinates(ima, im_coords, order=1)

  pano_im=pano_im / np.expand_dims((pano_im[:,:,3]+1e-6),2)
  pano_im = pano_im[:,:,0:3]

  return pano_im

# allow accessing these functions by render.*
render=types.SimpleNamespace()
render.pairwise_warp=pairwise_warp
render.pairwise_warp_file=pairwise_warp_file
render.render_spherical=render_spherical

### panorama.py

In [ ]:
# Copyright 2017 Google Inc.

# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at

# https://www.apache.org/licenses/LICENSE-2.0

# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

import os.path
from time import time
import numpy as np

#import im_util
#import match
#import geometry
#import ransac
#import render

class PanoramaStitcher:
  """
  Stitch a panorama from input images
  """
  def __init__(self, images, params={}):
    self.images = images
    num_images = len(images)
    self.matches = [[None]*num_images for _ in range(num_images)]
    self.num_matches = np.zeros((num_images, num_images))
    self.stitch_order = []
    self.R_matrices = [None]*num_images
    self.params = params
    self.params.setdefault('fov_degrees', 45)
    self.params.setdefault('draw_interest_points', False)
    self.params.setdefault('draw_matches', False)
    self.params.setdefault('draw_pairwise_warp', False)
    self.params.setdefault('results_dir', os.path.expanduser('~/results/tmp/'))

  def stitch(self):
    """
    Match images and perform alignment
    """
    self.match_images()
    self.align_panorama()

  def match_images(self):
    """
    Match images
    """
    im=match.ImageMatcher(self.params)
    self.matches, self.num_matches = im.match_images(self.images)

  def align_panorama(self):
    """
    Perform global alignment
    """
    # FORNOW identity rotations
    num_images = len(self.images)
    for i in range(num_images):
      self.R_matrices[i]=np.eye(3,3)

    """
    ***************************************************************
    *** TODO write code to compute a global rotation for each image
    ***************************************************************
    """


    """
    ***************************************************************
    """

  def render(self, render_params={}):
    """
    Render output panorama
    """
    print('[ render panorama ]')
    t0=time()
    P_matrices = self.get_projection_matrices()
    pano_im=render.render_spherical(self.images, P_matrices, render_params)
    t1=time()
    print(' % .2f secs' % (t1-t0))

    return pano_im

  def get_projection_matrices(self):
    """
    Return projection matrices P such that u~=PX
    """
    num_images = self.num_matches.shape[0]
    P_matrices = [None]*num_images
    for i in range(num_images):
      Ki = geometry.get_calibration(self.images[i].shape, self.params['fov_degrees'])
      P_matrices[i] = np.dot(Ki, self.R_matrices[i])

    return P_matrices


# allow accessing these functions by panorama.*
panorama=types.SimpleNamespace()
panorama.PanoramaStitcher=PanoramaStitcher

### Image Warping Test

The code below warps an image using a 3x3 transformation matrix. Experiment with the matrix P to test some of the different 2D transformations described in class, e.g., similarity, affine and projective transforms.

In [ ]:
# read image
image_dir='data/test'
im_filename1=image_dir+'/100-0038_img.jpg'
im=im_util.image_open(im_filename1)
im_rows,im_cols,_=im.shape

# set transformation matrix
P=[[1, 0.2, -64],
  [ 0, 1.1, -120],
  [ 0, 5.2e-4, 0.83]]

# warp coordinates
r0,r1=-im_rows/2, im_rows*3/2
c0,c1=-im_cols/2, im_cols*3/2
warp_rows, warp_cols=im_rows, im_cols

coords=im_util.coordinate_image(warp_rows,warp_cols,r0,r1,c0,c1)
coords_t=im_util.transform_coordinates(coords, P)

# visualise result
warp_im1=im_util.warp_image(im,coords)
warp_im2=im_util.warp_image(im,coords_t)
alpha=im_util.warp_image(np.ones((im_rows,im_cols,1)),coords_t)
result_im=warp_im2*alpha + 0.5*warp_im1*(1-alpha)

ax1=plt.subplot(1,1,1)
plt.imshow(result_im)
plt.axis('off')

### Interest Points Test

We will use the interest points and descriptors implemented in [Project 1](https://courses.cs.washington.edu/courses/csep576/18sp/projects/Project1.html "Project 1"). If you had trouble getting these to work, contact your TA.

Run the two code blocks below to check your interest points and descriptors are working. For subsequent steps to run well, you should aim for about 100-1000 interest points.

In [ ]:
"""
Read a pair of input images and convert to grey
"""
image_dir='data/test'
#im_filename1=image_dir+'/100-0023_img.jpg'
#im_filename2=image_dir+'/100-0024_img.jpg'
im_filename1=image_dir+'/100-0038_img.jpg'
im_filename2=image_dir+'/100-0039_img.jpg'

im1 = im_util.image_open(im_filename1)
im2 = im_util.image_open(im_filename2)

img1 = np.mean(im1, 2, keepdims=True)
img2 = np.mean(im2, 2, keepdims=True)

#optionally plot images
#ax1,ax2=im_util.plot_two_images(im1, im2)

"""
Find interest points in the image pair
"""
print('[ find interest points ]')
t0=time()
ip_ex = interest_point.InterestPointExtractor()
ip1 = ip_ex.find_interest_points(img1)
print(' found '+str(ip1.shape[1])+' in image 1')
ip2 = ip_ex.find_interest_points(img2)
print(' found '+str(ip2.shape[1])+' in image 2')
t1=time()
print(' % .2f secs ' % (t1-t0))

# optionally draw interest points
#print('[ drawing interest points ]')
#ax1,ax2=im_util.plot_two_images(im1,im2)
#interest_point.draw_interest_points_ax(ip1, ax1)
#interest_point.draw_interest_points_ax(ip2, ax2)

In [ ]:
"""
Extract and match descriptors
"""
print('[ extract descriptors ]')
t0=time()
desc_ex = interest_point.DescriptorExtractor()
desc1 = desc_ex.get_descriptors(img1, ip1)
desc2 = desc_ex.get_descriptors(img2, ip2)
t1=time()
print(' % .2f secs' % (t1-t0))

print('[ match descriptors ]')
t0=time()
match_idx = desc_ex.match_descriptors(desc1, desc2)
t1=time()
print(' % .2f secs' % (t1-t0))

ipm=ip2[:,match_idx]

print('[ drawing matches ]')
t0=time()
ax1,ax2=im_util.plot_two_images(im1,im2)
interest_point.draw_matches_ax(ip1, ipm, ax1, ax2)
t1=time()
print(' % .2f secs' % (t1-t0))

### RANSAC Implementation

We will now use RANSAC to find consistent matches.

First we will implement a test to count the number of matches consistent with a Similarity transform. The code below generates a random Similarity transform S and a random set of points x. It then transforms the points and adds noise, and checks to see how many of these points are consistent with the ground truth transformation S.

Open `ransac.py` and implement the function `consistent`. You should find a high fraction (~80% or more) points are consistent with the true Similarity transform S when running the code below.  

In [ ]:
"""
Test RANSAC functions using synthetic data
"""
# make a random S matrix
sd_pos=100
sd_angle=np.pi
theta=np.random.randn()*sd_angle
tx=np.random.randn()*sd_pos
ty=np.random.randn()*sd_pos
ct=np.cos(theta)
st=np.sin(theta)
S=[[ct,st,tx],[-st,ct,ty],[0,0,1]]

# generate random points
num_points=100
sd_points=20
x = np.random.randn(2,num_points)*sd_points
xh = geometry.hom(x)

# transform points and add noise
sd_noise=5
yh = np.dot(S, xh)
y = geometry.unhom(yh)
yn = y + np.random.randn(2,num_points)*sd_noise

print('[ Test of consistent ]')
rn = ransac.RANSAC()
inliers0=rn.consistent(S,x,yn)
num_consistent=np.sum(inliers0)
print(' number of points consistent with true S = '+str(num_consistent))
if (num_consistent > 0.75*num_points):
    print(' consistency check is working!')

Now select a sample of 2 point corresondences and compute the Similarity transform corresponding to this pair. Implement `compute_similarity` in `ransac.py` and run the code below to compute the number of inliers. Try varying the indices of the sample to see how the number of inliers varies.

In [ ]:
print('[ Test of compute_similarity ]')
sample=[0,1]
S1=rn.compute_similarity(x[:,sample],yn[:,sample])
inliers1=rn.consistent(S1,x,yn)
num_consistent=np.sum(inliers1)
print(' number of points consistent with sample S = '+str(num_consistent))

Finally, finish the implementation of RANSAC by completing `ransac_similarity` in `ransac.py`. When completed you should find most of the points are labelled consistent.

In [ ]:
print('[ Test of ransac_similarity ]')
S2, inliers2=rn.ransac_similarity(x, yn)
num_consistent=np.sum(inliers2)
print(' number of points consistent with ransac S = '+str(num_consistent))
if (num_consistent > 0.75*num_points):
    print(' ransac succeeded!')

We'll now move away from our synthetic test data and run the same code on the interest point matches obtained using the input image pair above. Review the code below and check that the output looks reasonable. You should see a good set of geometrically consistent matches.

In [ ]:
"""
Perform RANSAC on interest point matches
"""
print('[ do ransac ]')
t0=time()
rn = ransac.RANSAC()
S, inliers = rn.ransac_similarity(ip1,ipm)
t1=time()
num_inliers_s = np.sum(inliers)
print(' found '+str(num_inliers_s)+' matches')
print(' % .2f secs' % (t1-t0))

ip1c = ip1[:, inliers]
ipmc = ipm[:, inliers]

print('[ drawing matches ]')
t0=time()
ax1,ax2=im_util.plot_two_images(im1,im2)
interest_point.draw_matches_ax(ip1c, ipmc, ax1, ax2)
t1=time()
print(' % .2f secs' % (t1-t0))

# optionally plot descriptors for matched points
#inlier_id=np.flatnonzero(inliers)
#match_id=match_idx[inlier_id]
#interest_point.plot_matching_descriptors(desc1,desc2,inlier_id,match_id,plt)

### Rotation Estimation

The next task is to estimate the true rotation between the images. To do this, we'll take a guess at the field of view of our input images, and use a closed form algorithm to estimate the rotation. Open `geometry.py` and complete the implementation of `compute_rotation`. You should find that a large number of the matches are consistent with your rotation, and the pairwise warped images should look sensible. Try experimenting with the field of view parameter. What is the best field of view for these images? 

In [ ]:
"""
Estimate rotation matrix by least squares
"""
print('[ estimate rotation ]')
t0=time()
# Note: assume field of view of 45 degrees
fov_degrees=45
print(' assuming fov='+str(fov_degrees))
K1 = geometry.get_calibration(im1.shape, fov_degrees)
K2 = geometry.get_calibration(im2.shape, fov_degrees)
R,H = geometry.compute_rotation(ip1c, ipmc, K1, K2)

num_inliers_r = np.sum(rn.consistent(H, ip1, ipm))
print(' num consistent with rotation = '+str(num_inliers_r))
if (num_inliers_r>0.9 * num_inliers_s):
    print(' compute rotation succeeded!')
t1=time()
print(' % .2f secs' % (t1-t0))
    
print('[ test pairwise warp ]')
t0=time()
im1_w, im2_w = render.pairwise_warp(im1, im2, H)
_= im_util.plot_two_images(0.5*(im1+im2_w), 0.5*(im2+im1_w))
t1=time()
print(' % .2f secs' % (t1-t0))

The following code renders the aligned images in a spherical coordinate system. Check that the images are well aligned.

In [ ]:
"""
Render 2 images in spherical coordinates
"""
images=[im1,im2]
P1=K1
P2=np.dot(K2,R)
P_matrices=[P1,P2]

render_params={}
render_params['render_width']=800
render_params['theta_min']=-45
render_params['theta_max']=45
render_params['phi_min']=-30
render_params['phi_max']=30

print ('[ render aligned images ]')
t0=time()
pano_im=render.render_spherical(images, P_matrices, render_params)
t1=time()
print(' % .2f secs' % (t1-t0))

plt.plot()
plt.imshow(pano_im)
plt.axis('off')

Let's add more images! The method `PanormaStitcher` class in `panorama.py` takes a set of images as input and wraps the interest point and matching code in the method `match_images`. Take a look at this function and test it on a set of images using the code below.

In [ ]:
"""
Read a set of input images
"""

print('[ read images ]')
image_dir='data/test'
im_filenames=os.listdir(image_dir)
im_filenames=[image_dir+'/'+fname for fname in im_filenames]

#im_filenames=[]
#im_filenames.append(image_dir+'/100-0023_img.jpg')
#im_filenames.append(image_dir+'/100-0024_img.jpg')
#im_filenames.append(image_dir+'/100-0038_img.jpg')
#im_filenames.append(image_dir+'/100-0039_img.jpg')

images=[]
for fname in im_filenames:
  images.append(im_util.image_open(fname))

"""
Stitch images
"""
stitch_params={}
stitch_params['fov_degrees']=45

num_images = len(im_filenames)
print(' stitching '+str(num_images)+' images')

pano=panorama.PanoramaStitcher(images, stitch_params)
pano.match_images()

print(' num_matches=')
print(pano.num_matches)

Now write code to compute a rotation matrix for each image (the first image is assumed to be the identity rotation) by chaining together pairwise rotations. The code for this should go in `align_panorama` in `panorama.py`.

You can now use the `render` method to stich all images in spherical coordinates, as shown in the code below. 

In [ ]:
pano.align_panorama()

render_params={}
render_params['render_width']=800
render_params['theta_min']=-45
render_params['theta_max']=45
render_params['phi_min']=-30
render_params['phi_max']=30

pano_im = pano.render(render_params)

plt.plot()
plt.imshow(pano_im)
plt.axis('off')

### Testing and Improving the Panorama Stitcher

You should now have a complete implementation of a basic panorama stitcher. Try it out using a few different image sets and make a note of any issues/artifacts in the results. How could the results be improved? Write a list of possible improvements, and think of new features you might like to add. Now implement some of these improvements / new features and document your work in the notebook below.

In [ ]:
### TODO your improvements to the panorama stitcher